# Automate Event Planning


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

In [ ]:
import os
from utils import get_openai_api_key,get_serper_api_key

os.environ['OPENAI_API_KEY'] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [18]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [19]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    agent=logistics_manager
)

In [16]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [20]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=False
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [30]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "Singapore City",
    'tentative_date': "2025-06-16",
    'expected_participants': 500,
    'budget': 200000,
    'venue_type': "Conference Hall"
}

- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [31]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in Singapore City that meets criteria for Tech Innovation Conference.




# Agent: Venue Coordinator
## Thought: Thought: I need to search for venues in Singapore City that can accommodate a Tech Innovation Conference. The venue should have suitable capacity and facilities for such an event. I'll start by searching for "conference venues in Singapore for tech innovation conference."
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"conference venues in Singapore for tech innovation conference\"}"
## Tool Output: 
{'searchParameters': {'q': 'conference venues in Singapore for tech innovation conference', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "Singapore's Tech Hub: Innovative Venues for Tech Conferences", 'link': 'https://eventflare.io/expert-advice/singapore/singapore-s-tech-hub-innovative-venues-for-tech-conferences', 'snippet': 'From an event space in Singapore to more specialised conference venues in Singapore, here are some innovative spots worthy of your big ideas.', 'position': 1}, {

- Display the generated `venue_details.json` file.

In [32]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '1 Raffles Boulevard, Singapore 039593',
 'booking_status': 'Available for booking with flexible space configurations, '
                   'please contact venue for specific date availability',
 'capacity': 9000,
 'name': 'Suntec Singapore Convention & Exhibition Centre'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [34]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference 2024  
### Marketing Activities and Attendee Engagement Report

---

## Event Overview  
The Tech Innovation Conference 2024 is an exciting gathering aimed at technology professionals, innovators, marketers, and business leaders. The conference will showcase the latest advancements in AI, SaaS, cloud computing, and other cutting-edge tech sectors through keynote presentations, panel discussions, and networking sessions.

---

## Marketing Activities  

### 1. Target Audience Identification  
- Technology professionals (developers, engineers, IT leaders)  
- Marketing and product managers interested in tech innovations  
- Entrepreneurs and startups in technology space  
- Academics and researchers focused on emerging tech  

### 2. Personalized Marketing Campaigns  
- Segmented email marketing with tailored content highlighting topics relevant to each subgroup  
- Use of attendee personas to adapt messaging for maximum resonance  

### 3. Digital Marketing Channels  
- **Social Media Advertising:** Targeted ads on LinkedIn, Twitter, and Facebook focusing on professional groups and tech communities  
- **Content Marketing:** Blog articles, guest posts, and thought leadership pieces about the conference themes and speakers  
- **Event Website:** An engaging, mobile-friendly website featuring detailed agenda, speaker bios, testimonials, and easy registration process  
- **SEO Optimization:** Use of relevant keywords like “Tech Innovation Conference 2024,” “AI and Cloud computing events,” etc., to improve organic search visibility  

### 4. Engagement Incentives  
- Early bird registration discounts  
- Exclusive access to networking events or workshops for registered attendees  
- Gamification contests on social media with prizes to encourage shares and engagement  
- Limited-time group booking discounts to encourage team participation  

### 5. Partnerships and Influencer Outreach  
- Collaboration with technology blogs, podcasts, and influencers to promote the event  
- Engaging sponsors and partners for cross-promotion and increased visibility  

### 6. Event Reminders and Updates  
- Regular email countdowns and updates on speakers, session highlights, and special events  
- Use of push notifications via event app and social media to keep potential attendees informed  

---

## Attendee Engagement Strategy  

### Pre-Event  
- Interactive webinars introducing conference themes and speakers  
- Social media Q&A sessions and live chats to build excitement  
- Early networking opportunities through virtual meetups and forums  

### During Event  
- Real-time social media engagement with branded hashtags  
- Live polls and Q&A sessions to enhance participation  
- Dedicated virtual networking lounges and interest-based breakout rooms  

### Post-Event  
- Sharing of recorded sessions and key takeaways to maintain engagement  
- Gathering feedback through surveys to improve future events  
- Continued community building through social media groups and newsletters  

---

## Expected Outcomes  

- Engage and register at least 500 professional attendees from diverse technology sectors  
- Achieve high social media engagement with branded hashtag trending during event days  
- Build a sustained community for future Tech Innovation Conferences and activities  

---

## Conclusion  
By leveraging targeted personalized marketing, diversified digital channels, engaging incentives, and continuous interaction before, during, and after the event, the Tech Innovation Conference 2024 is poised to successfully attract and engage over 500 attendees, positioning itself as a premier event in the technology landscape.

---

For more information and to register, visit the official event website [Insert URL here].

---

This comprehensive marketing plan ensures wide reach, strong engagement, and a memorable experience for all participants.